# Convolutional Neural Networks

## Preparación de ambiente

### Carga de módulos

In [ ]:
# Misc
from warnings import filterwarnings

# Data wrangling
import numpy as np
import pandas as pd
from keras.datasets import mnist

# Data Visualization
from PIL import Image # pip install Pillow
import cufflinks as cf

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Modeling
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization

# Model performance
from sklearn.metrics import confusion_matrix, classification_report

# Environment setup
cf.go_offline()
filterwarnings("ignore")

### Funciones auxiliares

In [ ]:
def print_data(data):
    for row in data:
        print(''.join('{:3}'.format(value) for value in row))

## Data Wrangling

### Carga de datos

In [ ]:
(X_train, y_train), (X_val, y_val) = mnist.load_data()

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
df = pd.DataFrame(X_train.reshape((X_train.shape[0], 28*28)))

In [ ]:
df["digit"] = y_train

In [ ]:
df

### EDA

In [ ]:
for _ in range(10):
    sam = df.sample()
    print(sam["digit"].values)
    x = sam.drop(columns=["digit"]).to_numpy()
    print_data(x.reshape(28, 28))

In [ ]:
del df, sam, x

In [ ]:
for img, number in zip(X_train[:10], y_train[:10]):
    print(number)
    print_data(img)

In [ ]:
for img in X_train[:10]:
    display(Image.fromarray(img).resize((150, 150)))

### Preprocesamiento

#### Escalado

In [ ]:
sc = MinMaxScaler()

In [ ]:
X_train.dtype

In [ ]:
X_train.max()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2])).shape

In [ ]:
X_train = sc.fit_transform(X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2])))
X_val = sc.transform(X_val.reshape((X_val.shape[0], X_val.shape[1]*X_val.shape[2])))

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_val = X_val.reshape((X_val.shape[0], 28, 28, 1))

In [ ]:
X_train.shape

#### Preparación de la target

In [ ]:
y_train.shape

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [ ]:
X_train.dtype

In [ ]:
y_train.shape

In [ ]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

#### Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

In [ ]:
X_train.shape, X_test.shape, X_val.shape

In [ ]:
y_train.shape, y_test.shape, y_val.shape

## Modelado

### Arquitectura base

In [ ]:
model = Sequential()

### Capa convolucional

In [ ]:
Conv2D?

In [ ]:
X_train.shape[1:]

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=X_train.shape[1:]))

### Max Pooling

In [ ]:
MaxPooling2D?

In [ ]:
model.add(MaxPooling2D((2, 2)))

### Flatten

In [ ]:
Flatten?

In [ ]:
model.add(Flatten())

### Capas "ocultas"

In [ ]:
Dense?

In [ ]:
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))

In [ ]:
y_train.shape

In [ ]:
model.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
model.summary()

### Configuración del entrenamiento

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

### Evaluación de resultados

In [ ]:
loss, acc = model.evaluate(X_val, y_val)

In [ ]:
loss, acc

In [ ]:
results = pd.DataFrame(history.history)

In [ ]:
results.iplot()

### Preservación del modelo

In [ ]:
model.save("cnn_model.h5")

### Mejora no. 1

In [ ]:
BatchNormalization?

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
loss, acc = model.evaluate(X_val, y_val)

In [ ]:
loss, acc

### Mejora no. 2

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
loss, acc = model.evaluate(X_val, y_val)

In [ ]:
loss, acc

### Predicciones

In [ ]:
np.argmax(model.predict(X_val), axis=1)

### Matriz de confusión

In [ ]:
confusion_matrix(np.argmax(y_val, axis=1), np.argmax(model.predict(X_val), axis=1))

In [ ]:
cm = pd.DataFrame(confusion_matrix(np.argmax(y_val, axis=1), np.argmax(model.predict(X_val), axis=1)))

In [ ]:
cm.iplot(kind="heatmap", colorscale="blues")

### Clasification report

In [ ]:
print(classification_report(np.argmax(y_val, axis=1), np.argmax(model.predict(X_val), axis=1)))

#### Métricas

$ \displaystyle{recall = \frac{TP}{TP+FN} \qquad precision = \frac{TP}{TP+FP} \qquad accuracy = \frac{TP+TN}{TP+FP+TN+FN} \qquad f1 \;Score=\frac{2*precision*recall}{precision+recall}}$

### Preservación del modelo

In [ ]:
model.save("cnn_model.h5")